## 导入必要的库

In [1]:
import torch
import torch.nn.functional as F

## 实战案例一：手动提取矩阵对角线

假设有一个 $3 \times 3$ 的矩阵，我们要提取它的对角线元素 [0, 4, 8]。常规方法可能是 `torch.diagonal`，但我们看看 `as_strided` 怎么做。

In [2]:
# 1. 创建 3x3 矩阵
A = torch.arange(9).view(3, 3)
print(f"原始矩阵:\n{A}")
# 0 1 2
# 3 4 5
# 6 7 8

原始矩阵:
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])


### 分析内存布局

- 内存里是: [0, 1, 2, 3, 4, 5, 6, 7, 8]
- A 的 stride 是 (3, 1)

### 计算对角线的 Stride

- 对角线是: A[0,0] -> 0, A[1,1] -> 4, A[2,2] -> 8
- 观察物理内存索引: 0 -> 4 -> 8
- 每次跳跃距离 = 4 个元素
- 为什么是 4? 因为换一行要跳 3 (stride[0])，换一列要跳 1 (stride[1])。 3 + 1 = 4。

In [3]:
# 施法：提取对角线
diagonal = torch.as_strided(
    A,
    size=(3,),  # 只要 3 个元素
    stride=(4,),  # 每次跳 4 步
)

print(f"\n提取的对角线: {diagonal}")


提取的对角线: tensor([0, 4, 8])


### 验证零拷贝特性

In [4]:
print("修改对角线第一个元素...")
diagonal[0] = 999
print(f"查看原矩阵(零拷贝证明):\n{A}")

修改对角线第一个元素...
查看原矩阵(零拷贝证明):
tensor([[999,   1,   2],
        [  3,   4,   5],
        [  6,   7,   8]])


## 实战案例二：滑窗操作 (Sliding Window)

这是 `as_strided` 最著名的用法。假设有一个序列 [0, 1, 2, 3, 4]，我们想要做一个窗口大小为 3 的滑窗：

- 窗口1: [0, 1, 2]
- 窗口2: [1, 2, 3]
- 窗口3: [2, 3, 4]

思考：这看似复制了数据，实际上可以零拷贝实现。

In [5]:
# 1. 原始序列
x = torch.arange(5)  # [0, 1, 2, 3, 4]
# stride: (1,) -> 动一步跳1个元素

# 2. 目标形状
# 我们想要 3 个窗口，每个窗口长 3
target_size = (3, 3)

### 计算神奇的 Stride

- 第0维(窗口维): 从 "窗口1" 到 "窗口2" (即从 0 到 1)。内存跳几步？ -> 跳 1 步。
- 第1维(窗内维): 从 "窗口内的0" 到 "窗口内的1" (即从 0 到 1)。内存跳几步？ -> 也是跳 1 步！

In [6]:
target_stride = (1, 1)

# 施法
windows = torch.as_strided(x, size=target_size, stride=target_stride)

print(f"原始数据: {x}")
print(f"滑窗视图:\n{windows}")

原始数据: tensor([0, 1, 2, 3, 4])
滑窗视图:
tensor([[0, 1, 2],
        [1, 2, 3],
        [2, 3, 4]])


In [7]:
# 验证是否真的没复制
print(f"\n内存地址是否相同: {x.data_ptr() == windows.data_ptr()}")
# 看起来好像多了很多数据，其实只是同一段内存在被反复读取


内存地址是否相同: True


## 实战案例三：Unfold + im2col

我们要构造一个 4 维张量，形状为 (输出行, 输出列, 核高, 核宽)，即 (3, 3, 2, 2)。

### 思考 Stride 的设计

- Dim 0 (窗口向下移): 图片物理内存要跳过一行，即跳 4 个元素。
- Dim 1 (窗口向右移): 图片物理内存要跳过一列，即跳 1 个元素。
- Dim 2 (窗口内部向下): 在窗口内部下一行，其实就是原图的下一行，跳 4 个元素。
- Dim 3 (窗口内部向右): 在窗口内部下一列，其实就是原图的下一列，跳 1 个元素。

In [8]:
# 1. 模拟一张 4x4 的图片 (H=4, W=4)
img = torch.arange(1, 17).float().view(4, 4)
print(f"原始图片 (4x4):\n{img}")
# Stride: (4, 1) -> 换行跳4个元素，换列跳1个元素

原始图片 (4x4):
tensor([[ 1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12.],
        [13., 14., 15., 16.]])


In [9]:
# 2. 使用 as_strided 手动提取窗口
# 目标形状: (3, 3, 2, 2) -> (Out_H, Out_W, Kernel_H, Kernel_W)
H, W = img.shape
KH, KW = 2, 2
OH, OW = H - KH + 1, W - KW + 1  # 输出尺寸 3x3

# 原始 stride 是 (4, 1)
s0, s1 = img.stride()

# 构造 Magic Stride
# 关键点：Dim 0 和 Dim 2 的 stride 是一样的！Dim 1 和 Dim 3 的 stride 是一样的！
windows = torch.as_strided(
    img,
    size=(OH, OW, KH, KW),
    stride=(s0, s1, s0, s1),  # (4, 1, 4, 1)
)
print(windows.shape)  # 应该是 (3, 3, 2, 2)

torch.Size([3, 3, 2, 2])


In [10]:
print("\nas_strided 提取的窗口视图 (3, 3, 2, 2):")
print(windows[0, 0])  # 打印第一个窗口 (左上角)
print(windows[0, 1])  # 打印第二个窗口 (向右滑一步)


as_strided 提取的窗口视图 (3, 3, 2, 2):
tensor([[1., 2.],
        [5., 6.]])
tensor([[2., 3.],
        [6., 7.]])


### 拉直每个窗口 -> 变成 (9, 4) 矩阵

9 个窗口，每个窗口 4 个像素

In [11]:
# .t() 是为了适应 PyTorch unfold 的格式 (C*K*K, L)
# 注意：这里调用 contiguous() 会触发复制，因为 view 需要连续内存。
# 但在底层 GEMM 优化中，往往直接对 strided 内存操作或分块复制。

im2col_manual = windows.contiguous().view(-1, KH * KW).t()

print(f"\n手动实现的 im2col 矩阵 (4, 9):\n{im2col_manual}")
# 每一列代表一个 2x2 的窗口的数据


手动实现的 im2col 矩阵 (4, 9):
tensor([[ 1.,  2.,  3.,  5.,  6.,  7.,  9., 10., 11.],
        [ 2.,  3.,  4.,  6.,  7.,  8., 10., 11., 12.],
        [ 5.,  6.,  7.,  9., 10., 11., 13., 14., 15.],
        [ 6.,  7.,  8., 10., 11., 12., 14., 15., 16.]])


### 使用官方 API

In [12]:
# unfold 输入必须是 4 维 (N, C, H, W)
input_tensor = img.view(1, 1, 4, 4)

# kernel_size=2
unfold_out = F.unfold(input_tensor, kernel_size=2)

print(f"\n官方 F.unfold 输出 (1, 4, 9):\n{unfold_out}")
print(f"两者是否一致: {torch.allclose(im2col_manual, unfold_out[0])}")


官方 F.unfold 输出 (1, 4, 9):
tensor([[[ 1.,  2.,  3.,  5.,  6.,  7.,  9., 10., 11.],
         [ 2.,  3.,  4.,  6.,  7.,  8., 10., 11., 12.],
         [ 5.,  6.,  7.,  9., 10., 11., 13., 14., 15.],
         [ 6.,  7.,  8., 10., 11., 12., 14., 15., 16.]]])
两者是否一致: True


### 执行卷积 = 矩阵乘法

In [13]:
# 卷积核
kernel = torch.tensor([[1.0, 1.0], [1.0, 1.0]]).view(1, 4)  # 拉直成 1行4列

# 执行卷积 = 矩阵乘法
# (1, 4) x (4, 9) = (1, 9)
conv_res = kernel @ unfold_out[0]

# 变回图片形状 (3, 3)
output_img = conv_res.view(3, 3)

print(f"\n卷积结果:\n{output_img}")


卷积结果:
tensor([[14., 18., 22.],
        [30., 34., 38.],
        [46., 50., 54.]])


## 总结

- **as_strided 是魔术师的手法**：它通过设置重复的 stride（如 4, 1, 4, 1），让同一个物理像素在逻辑张量中出现多次（既属于窗口A，也属于窗口B），实现了零拷贝的窗口切分。

- **Unfold 是魔术的包装**：它封装了 stride 计算和 reshape 操作，直接输出 im2col 格式的矩阵（Columns）。

- **im2col 是最终效果**：通过上述操作，将复杂的卷积变成了简单的矩阵乘法（Kernel_Matrix @ Unfolded_Image），从而利用 GPU 强大的 Tensor Core 进行并行计算。